In [1]:
from os import environ
environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.9.0 pyspark-shell' 

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "512m").\
        getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f8280954-313b-4bc8-bd57-48e9e8666d38;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.9.0 in central
	found commons-io#commons-io;2.6 in central
	found org.glassfish.jaxb#txw2;2.3.2 in central
:: resolution report :: resolve 237ms :: artifacts dl 6ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.9.0 from central in [default]
	commons-io#commons-io;2.6 from central in [default]
	org.glassfish.jaxb#txw2;2.3.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search

In [3]:
# XML_PATH = 'enwiki-20221001-pages-articles-multistream6.xml-p958046p1483661'
# XML_PATH = 'enwiki-20211020-pages-articles-multistream.xml.bz2'
XML_PATH = "enwiki-20211020-pages-articles-multistream.xml"

In [4]:
df = spark.read.format("com.databricks.spark.xml").option("rowTag", "page").load(XML_PATH)

In [5]:
df.printSchema()

root
 |-- id: long (nullable = true)
 |-- ns: long (nullable = true)
 |-- redirect: struct (nullable = true)
 |    |-- _VALUE: string (nullable = true)
 |    |-- _title: string (nullable = true)
 |-- revision: struct (nullable = true)
 |    |-- comment: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |-- contributor: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _deleted: string (nullable = true)
 |    |    |-- id: long (nullable = true)
 |    |    |-- ip: string (nullable = true)
 |    |    |-- username: string (nullable = true)
 |    |-- format: string (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- minor: string (nullable = true)
 |    |-- model: string (nullable = true)
 |    |-- parentid: long (nullable = true)
 |    |-- sha1: string (nullable = true)
 |    |-- text: struct (nullable = true)
 |    |    |-- _VALUE: string (nullable = true)
 |  

In [6]:
from pyspark.sql.functions import col
df_body = df.select(col("revision.text._VALUE"), col("title"))
df_body.printSchema()

root
 |-- _VALUE: string (nullable = true)
 |-- title: string (nullable = true)



In [7]:
df_body.show()

+--------------------+--------------------+
|              _VALUE|               title|
+--------------------+--------------------+
|#REDIRECT [[Compu...| AccessibleComputing|
|{{short descripti...|           Anarchism|
|#REDIRECT [[Histo...|  AfghanistanHistory|
|#REDIRECT [[Geogr...|AfghanistanGeography|
|#REDIRECT [[Demog...|   AfghanistanPeople|
|#REDIRECT [[Commu...|AfghanistanCommun...|
|#REDIRECT [[Trans...|AfghanistanTransp...|
|#REDIRECT [[Afgha...| AfghanistanMilitary|
|#REDIRECT [[Forei...|AfghanistanTransn...|
|#REDIRECT [[Assis...| AssistiveTechnology|
|#REDIRECT [[Amoeb...|        AmoeboidTaxa|
|{{Short descripti...|              Autism|
|#REDIRECT [[Histo...|      AlbaniaHistory|
|#REDIRECT [[Demog...|       AlbaniaPeople|
|#REDIRECT [[As_We...|        AsWeMayThink|
|#REDIRECT [[Polit...|   AlbaniaGovernment|
|#REDIRECT [[Econo...|      AlbaniaEconomy|
|{{Short descripti...|              Albedo|
|#REDIRECT [[Afroa...|AfroAsiaticLanguages|
|#REDIRECT [[Const...|  Artifica

In [8]:
def get_artist_expresions():
    musician_flags = []
    with open('musician_occupancy_flags', 'r', encoding='utf8') as f:
        while flag := f.readline().rstrip():
            musician_flags.append("(?is)\| *?occupation.*?"+ flag +".*?\n")
    return "|".join(musician_flags)

In [9]:
expresion = get_artist_expresions()
df_artists = df_body.filter(df_body['_VALUE'].rlike(expresion))

In [10]:
df_artists.show()

+--------------------+--------------------+
|              _VALUE|               title|
+--------------------+--------------------+
|{{Short descripti...|     Abraham Lincoln|
|{{short descripti...|            Ayn Rand|
|{{Short descripti...|          Allan Dwan|
|{{short descripti...|             Algeria|
|{{short descripti...|       Aldous Huxley|
|{{short descripti...|    Andrei Tarkovsky|
|{{short descripti...|    Alfred Hitchcock|
|{{short descripti...|        Alfred Nobel|
|{{pp-semi-indef}}...|Alexander Graham ...|
|{{redirect|Kurosa...|      Akira Kurosawa|
|{{short descripti...|      A. E. van Vogt|
|{{Use mdy dates|d...|  Advanced Chemistry|
|{{Short descripti...|         A. A. Milne|
|{{short descripti...|       Alvin Toffler|
|{{short descripti...|     Agatha Christie|
|{{short descripti...|      Hercule Poirot|
|{{Short descripti...|         Miss Marple|
|{{short descripti...|      Allen Ginsberg|
|{{Short descripti...|        Aristophanes|
|{{short descripti...|          

In [11]:
from udf_infobox import parse_infobox
from udf_discography import parse_discography
from udf_awards import parse_awards

In [12]:
df_parsed_infobox = df_artists.withColumn("about", parse_infobox(col("_VALUE")))
df_parsed_infobox = df_parsed_infobox.withColumn("songs", parse_discography(col("_VALUE")))
df_parsed_infobox = df_parsed_infobox.withColumn("awards_list", parse_awards(col("_VALUE")))

In [13]:
df_parsed_infobox.show()

+--------------------+--------------------+--------------------+--------------------+-----------+
|              _VALUE|               title|               about|               songs|awards_list|
+--------------------+--------------------+--------------------+--------------------+-----------+
|{{Short descripti...|     Abraham Lincoln|{'image': 'Abraha...|                  []|         []|
|{{short descripti...|            Ayn Rand|{'name': 'Ayn Ran...|                  []|         []|
|{{Short descripti...|          Allan Dwan|{'name': 'Allan D...|                  []|         []|
|{{short descripti...|             Algeria|                  {}|                  []|         []|
|{{short descripti...|       Aldous Huxley|{'name': 'Aldous ...|                  []|         []|
|{{short descripti...|    Andrei Tarkovsky|{'name': 'Andrei ...|                  []|         []|
|{{short descripti...|    Alfred Hitchcock|{'honorific_prefi...|                  []|         []|
|{{short descripti..

In [14]:
sc = spark._jsc.sc() 
n_workers =  len([executor.host() for executor in sc.statusTracker().getExecutorInfos() ]) -1

print(n_workers)

2


In [15]:
cols = ["title", "about", "songs", "awards_list"]
df = df_parsed_infobox.select(*cols)

In [16]:
df.write.json("full_test.json")